In [20]:
import hashlib
import sys
import os

In [21]:
DEBUG = False

# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../") if DEBUG else os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [22]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [23]:
df_silver = extrair_dados_database("alunos_ufrj.parquet", "silver")

In [24]:
# Colunas da dimensão ALUNO
colunas_aluno = [
    "nomeCompleto",
    "matriculaDre",
    "sexo",
    "idadeIngressoCursoAtual",
    "dataNascimento",
    "periodoIngressoCursoAtual",
    "periodoIngressoUFRJ",
    "formaIngresso",
    "periodosIntegralizados",
    "modalidadeCota",
    "notaEnem",
]

# Seleção e remoção de duplicatas
d_aluno = df_silver[colunas_aluno].drop_duplicates().reset_index(drop=True)

In [25]:
# Criação do SK_D_ALUNO usando SHA-256
d_aluno["SK_D_ALUNO"] = d_aluno["matriculaDre"].apply(
    lambda matricula: hashlib.sha256(matricula.encode()).hexdigest()
)

# Reordenação das colunas
d_aluno = d_aluno[["SK_D_ALUNO"] + colunas_aluno]

# Padronizando nome das colunas
d_aluno = d_aluno.rename(columns={
    "nomeCompleto": "DS_NOME_ALUNO",
    "matriculaDre": "DS_MATRICULA_DRE",
    "sexo": "DS_SEXO",
    "idadeIngressoCursoAtual": "VL_IDADE_CURSO_ATUAL",
    "dataNascimento": "DT_NASCIMENTO",
    "periodoIngressoCursoAtual": "DS_PERIODO_INGRESSO_CURSO_ATUAL",
    "periodoIngressoUFRJ": "DS_PERIODO_INGRESSO_UFRJ",
    "formaIngresso": "DS_FORMA_INGRESSO",
    "periodosIntegralizados": "VL_PERIODOS_INTEGRALIZADOS",
    "modalidadeCota": "DS_MODALIDADE_COTA",
    "notaEnem": "VL_NOTA_ENEM"
})

In [26]:
# Salvar dados na camada Gold
carregar_dados_database(d_aluno, "d_aluno.parquet", "gold")